
# Borealis Quickstart

*Authors: Theodor Isacsson and Isaac De Vlugt*

In this quickstart, we will show how Borealis can be programmed in minutes.
The Borealis hardware, which showcased a quantum computational advantage as can be seen in Xanadu's Nature publication [1], 
is based on <a href="./3_Time_domain_photonic_circuits.ipynb">time-domain multiplexing (TDM)</a>;
a single squeezed-light source emits batches of 216 time-ordered squeezed-light 
pulses that interfere with one another with the help of optical delay loops, 
programmable beamsplitters, and phase shifters. To demonstrate a quantum computational
advantage, we chose a task that is proven to be computationally hard: <a href="./2_Gaussian_boson_sampling_and_the_Hafnian.ipynb">Gaussian Boson Sampling (GBS)</a>.
We will show you how to run the experiment!


Also, if you would like to learn more about Borealis and how to program it in more detail, check out our <a href="./4_Operating_Borealis_beginner_tutorial.ipynb">Borealis demo for beginners</a>.




To start off, we import the necessary packages. Then, by loading
a [`strawberryfields.Device`](https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.Device.html?highlight=strawberryfields%20device#strawberryfields.Device) object which contains relevant
and up-to-date information about Borealis, we can retrieve the device
by calling the [`strawberryfields.Device`](https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.Device.html?highlight=strawberryfields%20device#strawberryfields.Device)
property on the `BraketEngine` object.



In [ ]:
import strawberryfields as sf
import numpy as np
from braket.strawberryfields_plugin import BraketEngine

eng = BraketEngine("arn:aws:braket:us-east-1::device/qpu/xanadu/Borealis")
device = eng.device

A time-domain program is defined by arguments that each quantum
gate applies at each time bin, within the duration of the program.
We're going to use helper functions that will do some heavy lifting for us.
The `strawberryfields.tdm.borealis_gbs` function
will summarize all the necessary information for Strawberry Fields
and our hardware to implement your circuit, then
`strawberryfields.tdm.get_mode_indices`
gives us the number of concurrent modes ``N`` and the mode indices ``n``.




In [ ]:
from strawberryfields.tdm import borealis_gbs, get_mode_indices

gate_args_list = borealis_gbs(device, modes=216, squeezing="high")
delays = [1, 6, 36]
n, N = get_mode_indices(delays)

Finally, we can construct and run the circuit.




In [ ]:
from strawberryfields.ops import Sgate, Rgate, BSgate, MeasureFock

prog = sf.TDMProgram(N)

with prog.context(*gate_args_list) as (p, q):
    Sgate(p[0]) | q[n[0]]
    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]])
    MeasureFock() | q[0]

<div class="alert alert-block alert-warning">
<b>Caution:</b> The following code will result in <a href="https://aws.amazon.com/braket/pricing/">usage fees</a> for Borealis. Please only uncomment the cell if you are comfortable with the costs.</div>

In [ ]:
# shots = 10_000
# results = eng.run(prog, shots=shots, crop=True)
# print(results.samples)

Congratulations for successfully demonstrating quantum advantage!




## References

[1] Madsen, L.S., Laudenbach, F., Askarani, M.F. et al. “Quantum computational advantage with a programmable photonic processor”, [Nature 606, 75-81](https://www.nature.com/articles/s41586-022-04725-x), 2022.
